<h2 align="center"> Deploy Models with TensorFlow Serving and Docker</h2>

### Task 2: Load and Preprocess Data

In [28]:
%%writefile -a train.py
import os
import time
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

Appending to train.py


In [29]:
#Souce: https://www.kaggle.com/snap/amazon-fine-food-reviews/data
!head -n 5 train.csv

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
184502,B001BCVY4W,A1JMR1N9NBYJ1X,Mad Ethyl Flint,0,0,4,1228176000,Doesn't look like catfood!,"When you first open the can, it looks like something you would eat.  And no catfood smell! Nice sized chunks of chicken and vegetables in a lot of gravy.<br /><br />That being said, Ms Casiopia lapped up all the gravy and left the rest.  This however is not the product's fault as she has done this before with other catfoods<br /><br />I would have given it 5 stars, but since I won't be purchasing it, I gave it 4.  If your cat will eat chunks and vegetables, this product is for you.<br /><br />I have donated the remainder of the package to a less fortunate friend.<br /><br />Thank you."
182779,B0052LZ6XI,A2CVFBDRXYFZG9,vanostran,0,0,5,1335657600,Solid Mayo,"This is a solid mayo. Will not disappoint. At first I was concerned about consistency, but this is the real deal. Buy it. You will not regret

In [30]:
%%writefile -a train.py

def load_data(filename, num_samples):
    df=pd.read_csv(filename, usecols=[6,9], nrows=num_samples)
    df.columns=['Ratings','Title']
    
    text=df["Title"].tolist()
    text=[str(t).encode('ascii','replace') for t in text]
    text=np.array(text, dtype=object)[:]
    
    labels=df["Ratings"].tolist()
    labels=[1 if i>=4 else 0 if i ==3 else -1 for i in labels]
    labels=np.array(pd.get_dummies(labels), dtype=int)[:]
    
    return labels, text

Appending to train.py


In [31]:
t_lable, t_txt = load_data("train.csv",100)
print(t_lable)

[[0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


### Task 3: Build the Classification Model using TF Hub

In [32]:
%%writefile -a train.py

## https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1
## https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1
def get_model():
    hub_layer= hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", 
                       output_shape=[128],input_shape=[],dtype=tf.string, trainable=False,
                       name="input_layer")
    model=tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', name='output'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.summary()
    return model

Appending to train.py


### Task 4: Define Training Procedure

In [33]:
%%writefile -a train.py

def train(EPOCHS=5, BATCH_SIZE=32, TRAIN_FILE='train.csv', VAL_FILE='test.csv'):
    WORKDIR=os.getcwd()
    print("Getting Training/Validation datasets")
    y_train, x_train= load_data('train.csv', num_samples=100000)
    y_val, x_val = load_data('test.csv', num_samples=10000)
    
    print("Train the model")
    model=get_model()
    model.fit(x_train, y_train, batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=1,
             validation_data=(x_val, y_val),
             callbacks=[tf.keras.callbacks.ModelCheckpoint(
                                                         os.path.join(WORKDIR,'model_checkpoint'),
                                                         monitor='val_loss',
                                                            verbose=1,
                                                         save_best_model=True,
                                                         save_weights_only=False,
                                                         mode='auto',
                                                         )])
    return model

Appending to train.py


### Task 5: Train and Export Model as Protobuf

In [34]:
%%writefile -a train.py

def export_model(model, base_path='amazon_review/'):
    path=os.path.join(base_path,str(int(time.time())))
    tf.saved_model.save(model, path)

if __name__=="__main__":
    model=train()
    export_model(model)

Appending to train.py


In [35]:
test_string=["very bad book"]
model.predict(test_string)

array([[0.07223512, 0.03168595, 0.89607894]], dtype=float32)

### Task 6: Test Model

#### Negative Review:

#### Positive Review:

### Task : TensorFlow Serving with Docker

`docker pull tensorflow/serving`

`docker run -p 8500:8500 \
            -p 8501:8501 \
            --mount type=bind,\
            source=amazon_review/,\
            target=/models/amazon_review \
            -e MODEL_NAME=amazon_review \
            -t tensorflow/serving`

### Task : Setup a REST Client to Perform Model Predictions

#### Perform Model Prediction

##### Support for gRPC and REST

- TensorFlow Serving supports
    - Remote Procedure Protocal (gRPC)
    - Representational State Transfer (REST)
- Consistent API structures
- Server supports both standards simultaneously
- Default ports:
    - RPC: 8500
    - REST: 8501

#### Predictions via REST

- Standard HTTP POST requests
- Response is a JSON body with the prediction
- Request from the default or specific model

Default URI scheme:

`http://{HOST}:{PORT}/v1/models/{MODEL_NAME}`

Specific model versions:

`http://{HOST}:{PORT}/v1/models/{MODEL_NAME}[/versions/{MODEL_VERSION}]:predict`

In [13]:
# %%writefile tf_serving_rest_client.py
import json
import requests
import sys

def get_rest_url(model_name, host='127.0.0.1', port='8501', verb='predict', version=None):
    """ generate the URL path"""
    url = "http://{host}:{port}/v1/models/{model_name}".format(host=host, port=port, model_name=model_name)
    if version:
        url += 'versions/{version}'.format(version=version)
    url += ':{verb}'.format(verb=verb)
    return url


def get_model_prediction(model_input, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""

    url = get_rest_url(model_name)
    #In the row format, inputs are keyed to instances key in the JSON request.
    #When there is only one named input, specify the value of instances key to be the value of the input:
    data = {"instances": [model_input]}
    
    rv = requests.post(url, data=json.dumps(data))
    if rv.status_code != requests.codes.ok:
        rv.raise_for_status()
    
    return rv.json()['predictions']

if __name__ == '__main__':

    print("\nGenerate REST url ...")
    url = get_rest_url(model_name='amazon_review')
    print(url)
    
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = input()
        if sentence == ':q':
            break
        model_input = sentence
        model_prediction = get_model_prediction(model_input)
        print("The model predicted ...")
        print(model_prediction)


Generate REST url ...
http://127.0.0.1:8501/v1/models/amazon_review:predict

Enter an Amazon review [:q for Quit]


 gg


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

### Task : Setup a gRPC Client to Perform Model Predictions

Modified from [https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_client.py](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_client.py#L152)

#### Predictions via gRPC

More sophisticated client-server connections

- Prediction data has to be converted to the Protobuf format
- Request types have designated types, e.g. float, int, bytes
- Payloads need to be converted to base64
- Connect to the server via gRPC stubs

#### gRPC vs REST: When to use which API standard

- Rest is easy to implement and debug
- RPC is more network efficient, smaller payloads
- RPC can provide much faster inferences!

In [14]:
%%writefile tf_serving_grpc_client.py
import sys
import grpc
from grpc.beta import implementations
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2, get_model_metadata_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc


def get_stub(host='127.0.0.1', port='8500'):
    channel = grpc.insecure_channel('127.0.0.1:8500') 
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    return stub


def get_model_prediction(model_input, stub, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    request.inputs['input_input'].CopyFrom(tf.make_tensor_proto(model_input))
    response = stub.Predict.future(request, 5.0)  # 5 seconds
    return response.result().outputs["output"].float_val


def get_model_version(model_name, stub):
    request = get_model_metadata_pb2.GetModelMetadataRequest()
    request.model_spec.name = 'amazon_review'
    request.metadata_field.append("signature_def")
    response = stub.GetModelMetadata(request, 10)
    # signature of loaded model is available here: response.metadata['signature_def']
    return response.model_spec.version.value

if __name__ == '__main__':
    print("\nCreate RPC connection ...")
    stub = get_stub()
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = raw_input() if sys.version_info[0] < 3 else input()
        if sentence == ':q':
            break
        model_input = [sentence]
        model_prediction = get_model_prediction(model_input, stub)
        print("The model predicted ...")
        print(model_prediction)

Writing tf_serving_grpc_client.py
